In [132]:
import urllib.request
import requests
import pandas as pd
import numpy as np
from transformers import pipeline

In [133]:
import os
from os.path import join

In [134]:
os.environ["HOME"]

'/Users/k0tarak'

In [135]:
data_test_filename = join(os.environ["HOME"], "Datasets/yandex_data_for_llm.csv")
df = pd.read_csv(data_test_filename, sep=',', index_col=False, usecols=lambda column: column not in ['Unnamed: 0'])

In [136]:
df.head(5000)

,rating,text
0,5.0,Приятное место. Обедали компанией в субботу. М...
1,4.0,"Хорошее кафе, есть для детей игровой уголок и ..."
2,4.0,\n4 с минусом. Туалет это дичь просто. Если ты...
3,5.0,Прекрасное заведение и атмосфера. Все было оче...
4,5.0,"Вежливый персонал, быстрая скорость отдачи зак..."
...,...,...
4995,3.0,Посмотрели на супер отзывы и пришли сюда за Ку...
4996,5.0,Любимое место уже на протяжении многих лет ❤️ ...
4997,5.0,Очень вкусное и свежее пиво по замечательной ц...
4998,5.0,"Место не дешевое, но очень уютное. Прекрасный ..."


In [137]:
df['text'][0]

'Приятное место. Обедали компанией в субботу. Места были. Симпатичный и уютный интерьер. Кухня разная но в целом вкусная и оригинальная. Цены Москва, может процентов на 15 дешевле в среднем. Интересно, что официанты очень молодые, даже подростки, пожалуй. Их обслуживание оставило сложные впечатления...смесь раздражения и умиления. В общем рекомендую к посещению'

In [138]:
iam = ''
folder = ''

In [221]:
def get_request(text, command):
    return {
      "modelUri": "gpt://b1gv6dqj1vfsrufpk96p/yandexgpt/latest",
      "completionOptions": {
        "stream": False,
        "temperature": 1,
        "maxTokens": "2000"
      },
      "messages": [
        {
          "role": "system",
          "text": command
        },
        {
          "role": "user",
          "text": text
        }
      ]
    };

def get_header():
    return {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + iam,
        'x-folder-id': folder
    }

command = "Сократи текст до 200 символов сохранив смысл."


In [195]:
text = 'Приятное место. Обедали компанией в субботу. Места были. Симпатичный и уютный интерьер. Кухня разная но в целом вкусная и оригинальная. Цены Москва, может процентов на 15 дешевле в среднем. Интересно, что официанты очень молодые, даже подростки, пожалуй. Их обслуживание оставило сложные впечатления...смесь раздражения и умиления. В общем рекомендую к посещению'
########В субботу обедали компанией. Приятное заведение с симпатичным интерьером. Уютная атмосфера и вкусная кухня. Цены, как в Москве, на 15 % ниже.\n\nОбслуживание оставили смешанные чувства: молодые официанты вызывали и раздражение, и умиление. Рекомендую к посещению


In [222]:
r = requests.post('https://llm.api.cloud.yandex.net/foundationModels/v1/completion', json=get_request(text, command), headers=get_header())

In [150]:
r = requests.post('https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync', json=get_request(text, command), headers=get_header())

In [223]:
r

<Response [200]>

In [224]:
r.json()

{'result': {'alternatives': [{'message': {'role': 'assistant',
     'text': 'Были компанией в субботу, обедали в приятном месте. Симпатичный интерьер, вкусная кухня. Цены на 15% ниже московских. Официанты — подростки, обслуживание неоднозначное. Рекомендую к посещению.'},
    'status': 'ALTERNATIVE_STATUS_FINAL'}],
  'usage': {'inputTextTokens': '98',
   'completionTokens': '45',
   'totalTokens': '143'},
  'modelVersion': '07.03.2024'}}

In [175]:
resp = requests.get('https://llm.api.cloud.yandex.net/operations/'+ r.json()['id'], headers=get_header())

KeyError: 'id'

In [169]:
resp.json()

{'id': 'd7qljd2m1ren6djnc7e1',
 'description': 'Async GPT Completion',
 'createdAt': '2024-05-11T14:36:31Z',
 'createdBy': 'aje2l3p3tt2rr9h5qq3e',
 'modifiedAt': '2024-05-11T14:36:31Z',
 'done': False,
 'metadata': None}

In [158]:

print("Original text: " + text)
print("")
print("Generated text: " + resp.json()["response"]['alternatives'][0]['message']['text'])

Original text: Приятное место. Обедали компанией в субботу. Места были. Симпатичный и уютный интерьер. Кухня разная но в целом вкусная и оригинальная. Цены Москва, может процентов на 15 дешевле в среднем. Интересно, что официанты очень молодые, даже подростки, пожалуй. Их обслуживание оставило сложные впечатления...смесь раздражения и умиления. В общем рекомендую к посещению



KeyError: 'response'

In [74]:
df_transform = df.copy()

In [96]:
count = 1
def convert(text):
    global count

    try:
        if count % 50 == 0:
            print(count)
        count+=1;
        response = requests.post('https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync', json=get_request(text, command), headers=get_header())
        return response.json()['id']
    except Exception as e:
        print("An exception occurred:", e)

    return ""

In [ ]:

df_transform["req_id"] = df_transform["text"].apply(convert)

In [111]:
df_transform.tail(100)

,rating,text,req_id
4900,5.0,"Очень понравилось, отличное заведение!\nОчень ...",d7qodeaihmesl94cc68r
4901,1.0,"Отзыв именно по организатору ""Зеленая лампа"" в...",d7qqpf7jg1277latdltm
4902,5.0,"Уютно и приятно. Вежливый персонал , тепло. Ко...",d7qom00ua3kq8vo94u5b
4903,5.0,Вполне приемлемо для ранга столовых. В зале оч...,d7qi2p3uivpnih74cdvl
4904,5.0,"Хороший интерьер, вежливые персонал. Но ценник...",d7qvc9a3mrb1r6fvg58q
...,...,...,...
4995,3.0,Посмотрели на супер отзывы и пришли сюда за Ку...,d7q4fgje6f5aehqvu1p2
4996,5.0,Любимое место уже на протяжении многих лет ❤️ ...,d7quqlv85ppkv8vs7hsr
4997,5.0,Очень вкусное и свежее пиво по замечательной ц...,d7qja44vtt4jiocquitj
4998,5.0,"Место не дешевое, но очень уютное. Прекрасный ...",d7qo97i695n56i6n7i8e


In [125]:
count = 1

def get_result(id):
    global count
    if count % 50 == 0:
        print(count)

    count +=1
    try:
        response = requests.get('https://llm.api.cloud.yandex.net/operations/'+ id, headers=get_header())
        return response.json()["response"]['alternatives'][0]['message']['text']
    except Exception as e:
        print("An exception occurred:", e)

    return ""

In [ ]:
df_transform["text_converted"] = df_transform["req_id"].apply(get_result)

In [122]:
response = requests.get('https://llm.api.cloud.yandex.net/operations/'+ 'd7qodeaihmesl94cc68r', headers=get_header())
#response.json()["response"]['alternatives'][0]['message']['text']
response

<Response [200]>

In [ ]:
df_transform["text_converted"]

In [ ]:
filtered_df = df_transform.dropna(subset=['text_converted'])
filtered_df

In [131]:
import json

file_path = join(os.environ["HOME"], "Datasets/llm_antonyms.csv")

# Save the dictionary as a JSON file
filtered_df.to_csv(file_path, index=False)